Hi in this jupyter notebook we will setup 

**[6.1 YOLO](#6.1-YOLO)**<br>
&nbsp; &nbsp; &nbsp;[6.1.1 Set Up YOLO with DeepStream SDK](#6.1.1-Set-Up-YOLO-with-DeepStream-SDK)<br>
&nbsp; &nbsp; &nbsp;[6.1.2 Exercise: Run a YOLO Application](#6.1.2-Exercise:-Run-a-YOLO-Application)<br>
&nbsp; &nbsp; &nbsp;[6.1.3 Exercise: Annotate Your Own Video with YOLO](#6.1.3-Exercise:-Annotate-Your-Own-Video-with-YOLO)<br>
**[6.2 Faster-RCNN](#6.2-Faster-RCNN)**<br>
**[6.3 SSD](#6.3-SSD)**<br>

In [32]:
# Set some path locations for readability
YOLO_DETECTOR = '/opt/nvidia/deepstream/deepstream/sources/objectDetector_Yolo'
DLI_APPS = '/opt/nvidia/deepstream/deepstream/sources/deepstream_python_apps/dli_apps'
MY_APPS = '/opt/nvidia/deepstream/deepstream/sources/deepstream_python_apps/my_apps'

In [ ]:
# Set up the workspace for my new Python apps
MY_APPS = '/opt/nvidia/deepstream/deepstream/sources/deepstream_python_apps/my_apps'
!cp -r $PYTHON_APPS/common $MY_APPS/

In [33]:
# Explore the contents objectDetector_Yolo
# There are a number of configuration files to choose from
!ls -l $YOLO_DETECTOR

total 519972
-rw-r--r-- 1 root root      4741 Oct  5  2021 README
-rw-r--r-- 1 root root      3436 Oct  5  2021 config_infer_primary_yoloV2.txt
-rw-r--r-- 1 root root      3455 Oct  5  2021 config_infer_primary_yoloV2_tiny.txt
-rw-r--r-- 1 root root      3484 Oct  5  2021 config_infer_primary_yoloV3.txt
-rw-r--r-- 1 root root      3455 Oct  5  2021 config_infer_primary_yoloV3_tiny.txt
-rw-r--r-- 1 root root      4265 Oct  5  2021 deepstream_app_config_yoloV2.txt
-rw-r--r-- 1 root root      4270 Oct  5  2021 deepstream_app_config_yoloV2_tiny.txt
-rw-r--r-- 1 root root      4265 Oct  5  2021 deepstream_app_config_yoloV3.txt
-rw-r--r-- 1 root root      4270 Oct  5  2021 deepstream_app_config_yoloV3_tiny.txt
-rw-r--r-- 1 root root       624 Oct  5  2021 labels.txt
drwxr-xr-x 1 root root      4096 Oct 30 09:18 nvdsinfer_custom_impl_Yolo
-rwxr-xr-x 1 root root      2332 Oct  5  2021 prebuild.sh
-rw-r--r-- 1 root root      1488 Oct 30 08:06 yolov2-tiny.cfg
-rw-r--r-- 1 root root  44948600 Aug

In [10]:
# Take a look at the 80 objects that YOLO can detect
!cat $YOLO_DETECTOR/labels.txt

person
bicycle
car
motorbike
aeroplane
bus
train
truck
boat
traffic light
fire hydrant
stop sign
parking meter
bench
bird
cat
dog
horse
sheep
cow
elephant
bear
zebra
giraffe
backpack
umbrella
handbag
tie
suitcase
frisbee
skis
snowboard
sports ball
kite
baseball bat
baseball glove
skateboard
surfboard
tennis racket
bottle
wine glass
cup
fork
knife
spoon
bowl
banana
apple
sandwich
orange
broccoli
carrot
hot dog
pizza
donut
cake
chair
sofa
pottedplant
bed
diningtable
toilet
tvmonitor
laptop
mouse
remote
keyboard
cell phone
microwave
oven
toaster
sink
refrigerator
book
clock
vase
scissors
teddy bear
hair drier
toothbrush

## 6.1.1 Set Up YOLO with DeepStream SDK
The instructions for setting up YOLO for DeepStream are provided in the [README](deepstream/sources/objectDetector_Yolo/README) file in the `objectDetector_Yolo` folder. 

You must still download the YOLO weights and build the model.  Follow the steps below to set up YOLO on your Jetson Nano.  Your Jetson Nano must be connected to the Internet.
1. Download the weights and configuration for the model

In [8]:
# Download YOLO weights from the Internet
#!cd $YOLO_DETECTOR && ./prebuild.sh

yolov2.cfg          100%[===================>]   2.66K  --.-KB/s    in 0.005s  
yolov2.weights      100%[===================>] 194.49M   330KB/s    in 10m 33s 
yolov2-tiny.cfg     100%[===================>]   1.45K  --.-KB/s    in 0.004s  
yolov2-tiny.weights 100%[===================>]  42.87M   386KB/s    in 1m 51s  
yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0.02s   
yolov3.weights      100%[===================>] 236.52M   180KB/s    in 16m 6s  
yolov3-tiny.cfg     100%[===================>]   1.87K  --.-KB/s    in 0.02s   
yolov3-tiny.weights 100%[===================>]  33.79M   352KB/s    in 1m 42s  


2. Check the CUDA version on your machine

In [11]:
!cat /usr/local/cuda/version.txt

CUDA Version 10.2.300


3. If the CUDA Version does not match the variable CUDA_VER in the following cell, modify the value to match your CUDA version correctly.  Only do this to one decimal point, for example "CUDA_VER=10.2"
4. Execute the following cell to build the YOLO engine

In [9]:
# TODO Verify the CUDA_VER value
# Build YOLO engine
CUDA_VER = 10.2
!cd $YOLO_DETECTOR/nvdsinfer_custom_impl_Yolo/ \
    && CUDA_VER=$CUDA_VER make

g++ -c -o nvdsinfer_yolo_engine.o -Wall -std=c++11 -shared -fPIC -Wno-error=deprecated-declarations -I../../includes -I/usr/local/cuda-10.2/include nvdsinfer_yolo_engine.cpp
g++ -c -o nvdsparsebbox_Yolo.o -Wall -std=c++11 -shared -fPIC -Wno-error=deprecated-declarations -I../../includes -I/usr/local/cuda-10.2/include nvdsparsebbox_Yolo.cpp
g++ -c -o yoloPlugins.o -Wall -std=c++11 -shared -fPIC -Wno-error=deprecated-declarations -I../../includes -I/usr/local/cuda-10.2/include yoloPlugins.cpp
g++ -c -o trt_utils.o -Wall -std=c++11 -shared -fPIC -Wno-error=deprecated-declarations -I../../includes -I/usr/local/cuda-10.2/include trt_utils.cpp
trt_utils.cpp: In function 'nvinfer1::ILayer* netAddMaxpool(int, std::map<std::__cxx11::basic_string<char>, std::__cxx11::basic_string<char> >&, nvinfer1::ITensor*, nvinfer1::INetworkDefinition*)':
trt_utils.cpp:145:96: warning: 'nvinfer1::IPoolingLayer* nvinfer1::INetworkDefinition::addPooling(nvinfer1::ITensor&, nvinfer1::PoolingType, nvinfer1::DimsH

## 6.1.2 Exercise: Run a YOLO Application
Run the following sample application to see YOLO in action.  After running the following cells, DeepStream will save the result to `out.mp4`.  Download `out.mp4` by right clicking it in JupyterLab file browser and select "Download". Then play it on your machine.

In [34]:
# Check usage of your new test1 mp4 app with the help option
!cd $MY_APPS/dst-count-vehicles-mp4-out \
    && python3 deepstream_test1_mp4_out.py --help

usage: deepstream_test1_mp4_out.py [-h] -i INPUT [-c {H264,H265}] [-b BITRATE]
                                   [-o OUTPUT]

RTSP Output Sample Application Help

optional arguments:
  -h, --help            show this help message and exit
  -i INPUT, --input INPUT
                        Path to input H264 elementry stream
  -c {H264,H265}, --codec {H264,H265}
                        Streaming Codec H264/H265 , default=H264
  -b BITRATE, --bitrate BITRATE
                        Set the encoding bitrate
  -o OUTPUT, --output OUTPUT
                        Set the output file path


In [35]:
# Run the app
INPUT_PATH = '/dli/task/test4.h264'
OUTPUT_PATH = INPUT_PATH.replace('h264','mp4')
!cd $MY_APPS/dst-count-vehicles-mp4-out \
    && python3 deepstream_test1_mp4_out.py \
        -o $OUTPUT_PATH_EX2 \
        -i $INPUT_PATH

Creating Pipeline 
 
Creating Source 
 
Creating H264Parser 

Creating Decoder 

Creating H264 Encoder
Playing file /dli/task/test4.h264 
Unknown or legacy key specified 'is-classifier' for group [property]
Warn: 'threshold' parameter has been deprecated. Use 'pre-cluster-threshold' instead.
Warn: 'threshold' parameter has been deprecated. Use 'pre-cluster-threshold' instead.
Warn: 'threshold' parameter has been deprecated. Use 'pre-cluster-threshold' instead.
Warn: 'threshold' parameter has been deprecated. Use 'pre-cluster-threshold' instead.
Warn: 'threshold' parameter has been deprecated. Use 'pre-cluster-threshold' instead.
Adding elements to Pipeline 

Linking elements in the Pipeline 

Starting pipeline 

Opening in BLOCKING MODE 
Opening in BLOCKING MODE 
gstnvtracker: Loading low-level lib at /opt/nvidia/deepstream/deepstream/lib/libnvds_nvmultiobjecttracker.so
~~ CLOG[/dvs/git/dirty/git-master_linux/deepstream/sdk/src/utils/nvmultiobjecttracker/include/modules/NvMultiObjectTr

In [39]:
# Watch the saved video
import os
from IPython.display import Video

video_path = os.path.relpath(OUTPUT_PATH)
Video(video_path, width = 600, height = 300)

# Thank you!